# Strumento di Analisi Fisica Finanziaria
**Analisi di Mercato Avanzata utilizzando la Meccanica Lagrangiana e l'Elaborazione dei Segnali di Fourier**

Questo strumento applica principi fisici alle serie temporali finanziarie:
1.  **Analisi di Fourier**: Scompone i movimenti di prezzo in frequenze dominanti per proiettare potenziali cicli futuri.
2.  **Percorso di Minima Azione**: Calcola un percorso di prezzo "teorico" che minimizza l'Energia Cinetica (volatilità) e l'Energia Potenziale (deviazione dai fondamentali), creando un filtro di levigatura a zero lag.

---

In [5]:
# @title 0. (SOLO SE HAI ERRORI) Caricamento Manuale Librerie
# Esegui questa cella SOLO se ricevi "ModuleNotFoundError: No module named 'numpy'"
import sys
import os

# Tenta di trovare e aggiungere il virtual environment creato automaticamente
# Prova percorsi comuni per macOS/Linux
possible_paths = [
    "venv/lib/python3.9/site-packages",
    "venv/lib/python3.10/site-packages",
    "venv/lib/python3.11/site-packages",
    "venv/lib/python3.12/site-packages",
]
found = False
for p in possible_paths:
    venv_path = os.path.abspath(p)
    if os.path.exists(venv_path):
        if venv_path not in sys.path:
            sys.path.append(venv_path)
            print(f"Aggiunto path librerie: {venv_path}")
            found = True
            break

if not found:
    print("Nessun venv trovato automaticamente. Assicurati di aver lanciato setup_env.sh")

# Prova a installare con magic command (funziona in alcuni kernel)
try:
    import numpy
    print("Numpy trovato!")
    import plotly
    print("Plotly trovato!")
except ImportError:
    print("Librerie non trovate. Se stai usando il kernel 'Financial Physics', dovrebbero esserci.")
    print("Se sei sul kernel 'Python 3' standard, prova a eseguire: %pip install numpy pandas yfinance plotly scipy")

Aggiunto path librerie: /Users/andreavitofrancesco/Documents/analisi borsa (azione metodo)/venv/lib/python3.9/site-packages
Librerie non trovate. Se stai usando il kernel 'Financial Physics', dovrebbero esserci.
Se sei sul kernel 'Python 3' standard, prova a eseguire: %pip install numpy pandas yfinance plotly scipy


In [6]:
# @title 1. Setup Globale & Importazioni
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
import warnings

warnings.filterwarnings('ignore')
pd.options.plotting.backend = "plotly"

# Costanti di Configurazione
DEFAULT_TICKER = "SPY"
DEFAULT_START_DATE = "2020-01-01"

print("Librerie importate con successo.")

ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [ ]:
# @title 2. Modulo di Acquisizione Dati

class MarketData:
    """
    Gestisce il download e la pre-elaborazione dei dati finanziari.
    """
    def __init__(self, ticker, start_date=None, end_date=None):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.data = None

    def fetch(self):
        """
        Scarica i dati da Yahoo Finance.
        """
        print(f"Recupero dati per {self.ticker}...")
        self.data = yf.download(self.ticker, start=self.start_date, end=self.end_date, progress=False)
        
        if self.data.empty:
            raise ValueError(f"Nessun dato trovato per il ticker {self.ticker}")
            
        # Pulisce le colonne MultiIndex se presenti (comune nelle nuove versioni di yfinance)
        if isinstance(self.data.columns, pd.MultiIndex):
            self.data.columns = self.data.columns.get_level_values(0)
            
        self.data = self.data['Close'].dropna()
        self.data.name = self.ticker
        print(f"Caricati {len(self.data)} punti dati.")
        return self.data

# Esempio di utilizzo
# md = MarketData("AAPL", "2023-01-01")
# px = md.fetch()

In [ ]:
# @title 3. Modulo di Elaborazione Segnali (Fourier)

class FourierEngine:
    """
    Esegue l'Analisi Spettrale e la Generazione di Futuri Sintetici.
    """
    def __init__(self, price_series, top_k=8):
        self.px = price_series
        self.top_k = top_k
        self._fit()

    def _fit(self):
        """
        Scompone il segnale in spazio logaritmico, trend lineare e frequenze residue.
        """
        # 1. Trasformazione logaritmica per stabilità
        self.lp = np.log(self.px).astype(float)
        self.t = np.arange(len(self.lp))
        self.N = len(self.lp)
        
        # 2. Estrazione Trend Lineare
        self.coef = np.polyfit(self.t, self.lp.values, 1)
        self.trend = np.polyval(self.coef, self.t)
        self.resid = self.lp.values - self.trend
        
        # 3. FFT (Trasformata di Fourier Veloce)
        self.freqs = np.fft.rfftfreq(self.N, d=1.0)
        self.F = np.fft.rfft(self.resid)
        power = np.abs(self.F)
        
        # 4. Filtra le migliori TOP_K frequenze (ignora la componente DC all'indice 0)
        order = np.argsort(power[1:])[::-1][:self.top_k] + 1
        self.top_idx = np.sort(order)
        
        self.top_freqs = self.freqs[self.top_idx]
        self.top_amps = (2.0 / self.N) * np.abs(self.F[self.top_idx])
        self.top_phase = np.angle(self.F[self.top_idx])

    def reconstruct_scenario(self, future_horizon=60, amp_scale=1.0, phase_jitter=0.0, seed=None):
        """
        Genera un percorso sintetico estendendo di `future_horizon` giorni.
        """
        rng = np.random.default_rng(seed)
        
        # Vettore tempo: Passato + Futuro
        t2 = np.arange(self.N + future_horizon)
        
        # Estendi Trend Lineare
        trend2 = np.polyval(self.coef, t2)
        
        # Ricostruisci oscillazione dalle componenti coseno
        phases = self.top_phase.copy()
        if phase_jitter > 0:
            phases += rng.normal(0.0, phase_jitter, size=len(phases))
            
        resid2 = np.zeros_like(t2, dtype=float)
        for A, w, ph in zip(self.top_amps * amp_scale, 2 * np.pi * self.top_freqs, phases):
            resid2 += A * np.cos(w * t2 + ph)
            
        # Ri-esponenziazione
        lp2 = trend2 + resid2
        px2 = np.exp(lp2)
        
        # Genera Indice Temporale corretto
        try:
            last_date = self.px.index[-1]
            # Inferisci frequenza o default a giorno lavorativo (B)
            freq = pd.infer_freq(self.px.index)
            if not freq: freq = 'B'
            
            future_dates = pd.date_range(last_date, periods=future_horizon + 1, freq=freq)[1:]
            full_idx = self.px.index.append(future_dates)
        except:
            # Fallback a RangeIndex se le date falliscono
            full_idx = pd.RangeIndex(len(px2))
            
        return pd.Series(px2, index=full_idx)

    def get_components(self):
        """
        Restituisce un dataframe dei cicli dominanti trovati.
        """
        period_bars = (1 / np.maximum(self.top_freqs, 1e-12)).astype(int)
        return pd.DataFrame({
            "Frequenza": self.top_freqs,
            "Periodo (Barre)": period_bars,
            "Ampiezza": self.top_amps,
            "Fase (rad)": self.top_phase
        }).sort_values("Ampiezza", ascending=False)

# Esempio di utilizzo
# fe = FourierEngine(px, top_k=5)
# scen = fe.reconstruct_scenario(60)
# fe.get_components()

In [ ]:
# @title 4. Modulo Percorso di Minima Azione (Motore Fisico)

class ActionPath:
    """
    Calcola la traiettoria di 'Minima Azione' risolvendo un problema di minimizzazione Lagrangiana.
    Azione J = sum [ BETA * (x - F)^2 + ALPHA * (dx/dt)^2 ]
    """
    def __init__(self, price_series, alpha=1.0, beta=1.0, lookback_span=20):
        self.px = price_series
        self.alpha = float(alpha)
        self.beta = float(beta)
        self.span = lookback_span
        self._compute()

    def _compute(self):
        """
        Risolve il sistema matriciale tridiagonale per trovare x*.
        """
        # 1. Campo Fondamentale F (EWMA)
        # Usando adjust=False per corrispondere alla logica del notebook di riferimento
        self.F = self.px.ewm(span=self.span, adjust=False).mean()
        F_vals = self.F.values.astype(float)
        n = len(F_vals)
        
        if n < 3:
            raise ValueError("Serie temporale troppo corta per il calcolo della Minima Azione.")

        # 2. Costruisci Sistema Tridiagonale
        # Diagonale: B + 2A (eccetto estremità: B + A)
        # Fuori Diagonale: -A
        A, B = self.alpha, self.beta
        
        lower = np.full(n-1, -A, dtype=float)
        diag  = np.full(n,   B + 2*A, dtype=float)
        diag[0] = B + A
        diag[-1] = B + A
        upper = np.full(n-1, -A, dtype=float)
        rhs   = B * F_vals

        # 3. Risolvi (Algoritmo di Thomas)
        self.x_star_vals = self._solve_tridiag(lower, diag, upper, rhs)
        self.px_star = pd.Series(self.x_star_vals, index=self.px.index, name="px_star")

        # 4. Calcola Densità di Energia
        self.dX = self.px_star.diff().fillna(0.0)
        self.kin_density = 0.5 * A * (self.dX**2)                 # Cinetica ~ Volatilità
        self.pot_density = 0.5 * B * ((self.px_star - self.F)**2) # Potenziale ~ Deviazione
        
        self.action_density = self.kin_density + self.pot_density
        self.cumulative_action = self.action_density.cumsum()

    def _solve_tridiag(self, a, b, c, d):
        """
        Risolutore TDMA (algoritmo di Thomas).
        a: inferiore, b: diagonale, c: superiore, d: rhs
        """
        n = len(b)
        c_ = np.zeros(n-1, dtype=float)
        d_ = np.zeros(n, dtype=float)
        b_ = b.astype(float).copy()
        
        # Forward sweep
        c_[0] = c[0] / b_[0]
        d_[0] = d[0] / b_[0]
        for i in range(1, n-1):
            denom = b_[i] - a[i-1]*c_[i-1]
            c_[i] = c[i] / denom
            d_[i] = (d[i] - a[i-1]*d_[i-1]) / denom
            
        # Ultimo elemento
        d_[n-1] = (d[n-1] - a[n-2]*d_[n-2]) / (b_[n-1] - a[n-2]*c_[n-2])
        
        # Back substitution
        x = np.zeros(n, dtype=float)
        x[-1] = d_[n-1]
        for i in range(n-2, -1, -1):
            x[i] = d_[i] - c_[i]*x[i+1]
            
        return x

# Esempio di utilizzo
# ap = ActionPath(px, alpha=100, beta=1)
# ap.px_star.plot()

In [ ]:
# @title 5. Dashboard & Analisi (Esegui Questo!)
# --- CONFIGURAZIONE ---
TICKER = "SPY" 
ALPHA = 200.0   # Inerzia (Alto = Percorso più fluido/pesante)
BETA = 1.0      # Fedeltà (Alto = Più vicino ai Fondamentali)
TOP_K = 5       # Numero di frequenze di Fourier
FORECAST_H = 60 # Giorni futuri da proiettare
# ---------------------

try:
    # 1. Recupera Dati
    md = MarketData(TICKER, "2023-01-01")
    px = md.fetch()

    # 2. Esegui Motore Fisico
    mechanics = ActionPath(px, alpha=ALPHA, beta=BETA)
    px_star = mechanics.px_star
    
    # 3. Esegui Motore Fourier (per proiezione)
    fourier = FourierEngine(px, top_k=TOP_K)
    future_scenario = fourier.reconstruct_scenario(future_horizon=FORECAST_H, amp_scale=1.0)

    # 4. Costruisci Grafici
    fig = make_subplots(
        rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.08,
        specs=[[{"secondary_y": False}], [{"secondary_y": True}], [{"secondary_y": False}]],
        subplot_titles=("Prezzo vs Percorso Minima Azione vs Futuro Sintetico", 
                       "Densità di Energia & Azione Cumulata", 
                       "Cicli di Fourier Dominanti")
    )

    # -- RIGA 1: Analisi Prezzo --
    # Prezzo Reale
    fig.add_trace(go.Scatter(x=px.index, y=px.values, name="Prezzo Reale", mode="lines", line=dict(color="black", width=2)), row=1, col=1)
    # Percorso Fisico
    fig.add_trace(go.Scatter(x=px_star.index, y=px_star.values, name="Percorso Min-Azione", mode="lines", line=dict(color="#00cc96", width=2, dash="dash")), row=1, col=1)
    # Fondamentali
    fig.add_trace(go.Scatter(x=mechanics.F.index, y=mechanics.F.values, name="Fondamentali (Campo)", mode="lines", line=dict(color="blue", width=1, dash="dot")), row=1, col=1)
    # Proiezione Futura
    future_only = future_scenario[future_scenario.index > px.index[-1]]
    fig.add_trace(go.Scatter(x=future_only.index, y=future_only.values, name="Previsione Fourier", mode="lines", line=dict(color="magenta", width=2, dash="dot")), row=1, col=1)

    # -- RIGA 2: Analisi Energia --
    fig.add_trace(go.Scatter(x=mechanics.kin_density.index, y=mechanics.kin_density.values, name="Densità Cinetica", mode="lines", line=dict(width=1), fill='tozeroy', opacity=0.3), row=2, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=mechanics.pot_density.index, y=mechanics.pot_density.values, name="Densità Potenziale", mode="lines", line=dict(width=1), fill='tozeroy', opacity=0.3), row=2, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=mechanics.cumulative_action.index, y=mechanics.cumulative_action.values, name="Azione Cumulata", mode="lines", line=dict(color="black", width=2)), row=2, col=1, secondary_y=True)

    # -- RIGA 3: Componenti Fourier --
    comps = fourier.get_components().head(TOP_K)
    fig.add_trace(go.Bar(x=comps["Periodo (Barre)"].astype(str) + "d", y=comps["Ampiezza"], name="Ampiezza Ciclo"), row=3, col=1)

    fig.update_layout(height=1000, title_text=f"Analisi Fisica Finanziaria: {TICKER}", showlegend=True, hovermode="x unified")
    fig.show()

    print("Analisi Completata.")
    display(comps)

except Exception as e:
    print(f"Errore durante l'esecuzione: {e}")